In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import json

In [ ]:
filename = os.path.join("logs.json")

with open(filename, 'r') as file:
    json_list = json.load(file)


In [ ]:
timestamp_granularity = 2

min_x = None
max_x = None
min_y = None
max_y = None

events_at_timestamp = dict()
total_events = 0
for obj in json_list:
    if obj["eventType"] == "DEVICE_LOCATION_UPDATE":
        total_events += 1
        device_location_update = obj["deviceLocationUpdate"]
        pos_x = device_location_update["xPos"]
        pos_y = device_location_update["yPos"]
        if min_x == None or min_x > pos_x:
            min_x = pos_x

        if max_x == None or max_x < pos_x:
            max_x = pos_x

        if min_y == None or min_y > pos_y:
            min_y = pos_y

        if max_y == None or max_y < pos_y:
            max_y = pos_y

        if timestamp_granularity > 0:
            ts = int(str(obj["recordTimestamp"])[:-timestamp_granularity])
        else:
            ts = obj["recordTimestamp"]
        if ts not in events_at_timestamp:
            events_at_timestamp[ts] = []
        events_at_timestamp[ts].append(obj)

print(f"{len(events_at_timestamp)}/{total_events}")
print(events_at_timestamp)
print(f"x: {min_x} - {max_x}")
print(f"y: {min_y} - {max_y}")

In [ ]:
all_timestamps = list(events_at_timestamp.keys())

sampled_timestamp = all_timestamps[21]

location_precision = 50

data = np.zeros((int(max_x) // location_precision + 1, int(max_y) // location_precision + 1))

for event in events_at_timestamp[sampled_timestamp]:
    x = int(event["deviceLocationUpdate"]["xPos"]) // location_precision
    y = int(event["deviceLocationUpdate"]["yPos"]) // location_precision
    data[x][y] += 1

data = np.where(data == 0, np.nan, data)


# Create the heatmap
sns.heatmap(data, annot=True, cmap='coolwarm')

# Add labels and title
plt.xlabel('X Axis')
plt.ylabel('Y Axis')
plt.title('Heatmap Example')

# Show the plot
plt.show()